In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataset_dir = '/content/drive/MyDrive/IndoorOutdoorDataset'

In [ ]:
img_width, img_height = 224, 224
batch_size = 32
epochs = 10

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    directory=dataset_dir,
    labels='inferred',
    label_mode='binary',
    image_size=(img_width, img_height),
    batch_size=batch_size,
    validation_split=0.2,
    seed=42,
    shuffle=True,
    subset='training',
)

Found 800 files belonging to 2 classes.
Using 640 files for training.


In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
    directory=dataset_dir,
    labels='inferred',
    label_mode='binary',
    image_size=(img_width, img_height),
    batch_size=batch_size,
    validation_split=0.2,
    seed=42,
    shuffle=True,
    subset='validation',
)

Found 800 files belonging to 2 classes.
Using 160 files for validation.


In [ ]:
model = models.Sequential()
model.add(ResNet50V2(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3), classes=2))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

94668760/94668760 [==============================] - 1s 0us/step


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, 7, 7, 2048)        23564800  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 256)               25690368  
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 49296385 (188.05 MB)
Trainable params: 492

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_ds,
    epochs=epochs,
    validation_data=val_ds,
)

Epoch 1/10
20/20 [==============================] - 331s 15s/step - loss: 0.8096 - accuracy: 0.8281 - val_loss: 287.5085 - val_accuracy: 0.6625
Epoch 2/10
20/20 [==============================] - 293s 15s/step - loss: 0.2809 - accuracy: 0.9172 - val_loss: 3023.7878 - val_accuracy: 0.5063
Epoch 3/10
20/20 [==============================] - 288s 14s/step - loss: 0.4080 - accuracy: 0.8813 - val_loss: 635739.0000 - val_accuracy: 0.4938
Epoch 4/10
20/20 [==============================] - 292s 15s/step - loss: 0.5737 - accuracy: 0.8906 - val_loss: 267278.7500 - val_accuracy: 0.4938
Epoch 5/10
20/20 [==============================] - 292s 15s/step - loss: 0.9304 - accuracy: 0.8172 - val_loss: 17758412.0000 - val_accuracy: 0.5063
Epoch 6/10
20/20 [==============================] - 293s 15s/step - loss: 0.6164 - accuracy: 0.8031 - val_loss: 439.1665 - val_accuracy: 0.6062
Epoch 7/10
20/20 [==============================] - 285s 14s/step - loss: 0.4225 - accuracy: 0.8687 - val_loss: 3695.1941 - 

In [ ]:
model.save('/content/drive/MyDrive/resnet50v2sequence_indoor_outdoor_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
